In [ ]:
import sys
from pathlib import Path


# Detect project root correctly:
# Notebook CWD = /project-root/notebooks → parent = /project-root
project_root = Path.cwd().parent
print("Detected project root:", project_root)

# Add project root to Python import path (so vector_pipeline can be imported cleanly)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("Added to sys.path.")

Detected project root: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data


In [4]:

from vector_pipeline import settings 

# Update settings to absolute paths (fixing the notebooks-relative problem)
settings.DATA_PATH = project_root / "data" / "processed_product.pkl"
settings.CHROMA_DIR = project_root / "chroma_db"

print("DATA_PATH:", settings.DATA_PATH.resolve(), "-> exists:", settings.DATA_PATH.exists())
print("CHROMA_DIR:", settings.CHROMA_DIR.resolve())

DATA_PATH: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/data/processed_product.pkl -> exists: True
CHROMA_DIR: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db


In [5]:
from vector_pipeline.ingestion import build_or_load_vectorstore

print("Loading / building vectorstore...")
vs = build_or_load_vectorstore()
print("Vectorstore ready ✅")

Loading / building vectorstore...
Loading existing Chroma DB from /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db ...
Vectorstore loaded successfully.
Vectorstore ready ✅


In [6]:
import json

def print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

In [7]:
api_request = {
    "session_id": "S12345",
    "user_id": "U9001",
    "messages": [
        {"role": "user", "content": "Show me wireless headphones with noise cancelling"}
    ],
    "top_k": 4,
    "use_reranker": True
}

print("📥 Incoming API request:")
print_json(api_request)

📥 Incoming API request:
{
  "session_id": "S12345",
  "user_id": "U9001",
  "messages": [
    {
      "role": "user",
      "content": "Show me wireless headphones with noise cancelling"
    }
  ],
  "top_k": 4,
  "use_reranker": true
}


In [10]:
from vector_pipeline.api_wrapper import run_chat

result = run_chat(
    messages = list(api_request["messages"]),
    k=api_request.get("top_k", 5),
    use_reranker=api_request.get("use_reranker", True)
)

api_response = {
    "status": "success",
    "session_id": api_request["session_id"],
    "user_id": api_request["user_id"],
    "answer": result["answer"],
    "messages": result["messages"],  # updated history
    "retrieved": result["retrieved"]
}

print("📤 API response:")
print_json(api_response)

Loading existing Chroma DB from /home/viktoriia/Documents/ML/project-25-2-scrum-team-data/chroma_db ...
Vectorstore loaded successfully.
📤 API response:
{
  "status": "success",
  "session_id": "S12345",
  "user_id": "U9001",
  "answer": "Use ONLY the following product information to answer the question.\nIf the answer is not in the context, say you don't know.\n\nCONTEXT:\nProduct Title: Hollyland Lark M1 Wireless Lavalier Microphone with Noise Cancellation, 656ft Transmission,8H Battery Life, Compact Wireless Lapel Mic for DSLR Cameras/iPhone/Android/Live Streaming- Light Purple\nCategories: Musical Instruments, Microphones & Accessories, Microphones, Wireless Microphones & Systems, Wireless Lavalier Microphones\n\n---\n\nFeatures: 【Noise Cancellation & CD-level Audio Quality】This professional microphone has a built-in noise reduction chip, more effectively identifying the original sound in noisy environments. Wireless Lavalier mic has omnidirectional sound reception technology and r

In [11]:
history = [
    {"role": "user", "content": "Recommend a gaming laptop"},
]

session_id = "S777"
user_id = "U007"

print("🧩 First turn — incoming:")
print_json({"session_id": session_id, "user_id": user_id, "messages": history})

result1 = run_chat(history)

print("\n🤖 First answer:")
print(result1["answer"])

print("\n📤 API response (turn 1):")
print_json({
    "status": "success",
    "session_id": session_id,
    "user_id": user_id,
    "answer": result1["answer"],
    "messages": result1["messages"],
    "retrieved": result1["retrieved"]
})


🧩 First turn — incoming:
{
  "session_id": "S777",
  "user_id": "U007",
  "messages": [
    {
      "role": "user",
      "content": "Recommend a gaming laptop"
    }
  ]
}

🤖 First answer:
Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
4 or Celeron-compatible CPU or faster (multicore CPU recommended); 1 GB RAM (2 GB recommended); Windows 10, Windows 8, Windows 7, Windows Vista, or Windows XP; Windows-compatible sound card (ASIO driver support recommended); QuickTime recommended. Available USB port.

---

Features: 【Portability 】The usb podcast microphone is a portable mini computer mic for travel and portability. Gaming microphone is flexibility adjustable gooseneck design, tiny enough to take up only minimal desk space. The flexible PC microphone is ideal for everyday use in both busy broadcast and live environments. 【Compatibility】USB microphone features a standard USB connection, plug n' play for

In [12]:
history2 = result1["messages"] + [
    {"role": "user", "content": "What about something cheap but still good?"}
]

print("🧩 Second turn — incoming:")
print_json({"session_id": session_id, "user_id": user_id, "messages": history2})

result2 = run_chat(history2)

print("\n🤖 Second answer:")
print(result2["answer"])

print("\n📤 API response (turn 2):")
print_json({
    "status": "success",
    "session_id": session_id,
    "user_id": user_id,
    "answer": result2["answer"],
    "messages": result2["messages"],
    "retrieved": result2["retrieved"]
})


🧩 Second turn — incoming:
{
  "session_id": "S777",
  "user_id": "U007",
  "messages": [
    {
      "role": "user",
      "content": "Recommend a gaming laptop"
    },
    {
      "role": "assistant",
      "content": "Use ONLY the following product information to answer the question.\nIf the answer is not in the context, say you don't know.\n\nCONTEXT:\n4 or Celeron-compatible CPU or faster (multicore CPU recommended); 1 GB RAM (2 GB recommended); Windows 10, Windows 8, Windows 7, Windows Vista, or Windows XP; Windows-compatible sound card (ASIO driver support recommended); QuickTime recommended. Available USB port.\n\n---\n\nFeatures: 【Portability 】The usb podcast microphone is a portable mini computer mic for travel and portability. Gaming microphone is flexibility adjustable gooseneck design, tiny enough to take up only minimal desk space. The flexible PC microphone is ideal for everyday use in both busy broadcast and live environments. 【Compatibility】USB microphone features a sta

In [13]:
print(f"📚 Retrieved chunks: {len(result2['retrieved'])}")

for i, chunk in enumerate(result2["retrieved"], start=1):
    print(f"\n--- Chunk {i} ---")
    print("Metadata:", chunk["metadata"])
    print("Snippet:", chunk["snippet"][:250], "...")


📚 Retrieved chunks: 4

--- Chunk 1 ---
Metadata: {'product_title': 'Product Title: co2CREA Hard Case replacement for AKAI Professional MPK Mini MK3 / MPK Mini Play MK3 25 Key USB MIDI Keyboard Controller', 'product_id': 'B09ZNNYMQ9', 'color': 'All Black Case', 'features': 'Features: co2CREA hard travel case replacement for Akai professional mpk mini mk3 midi keyboard sound board controller or Akai mpk mini play mk3 25 key usb midi keyboard controller. Hard EVA shell case is shockproof dustproof and water resistance, protect your Akai mpk mini mk3 mini keyboard piano Akai midi keyboard beat machine from drops, scratches, bumps, splash, and dust. This Akai mpk mini mk3 mini piano synthesizer keyboard protection case perfect for your Akai mpk mini mk3 midi keyboard sound board mini piano keyboard or Akai mpk mini play mk3 25 key usb midi keyboard computer recording midi controllers. And this Akai mpk mini mk2 mini midi keyboard case has more space to accommodate the Akai professional mpk 

In [ ]:
def chat_once(session_id, user_id, user_input, history=None):
    if history is None:
        history = [{"role": "user", "content": user_input}]
    else:
        history = history + [{"role": "user", "content": user_input}]
    
    # simulate backend request
    req = {
        "session_id": session_id,
        "user_id": user_id,
        "messages": history,
    }
    
    res = run_chat(req["messages"])
    
    print("\n🗣️ User:", user_input)
    print("🤖 Bot:", res["answer"])
    return res["messages"]

# Example usage:
history = chat_once("S999", "UABC", "What is a good smartphone for photography?")
history = chat_once("S999", "UABC", "And something cheaper?", history)
history = chat_once("S999", "UABC", "Which one has best battery?", history)
